# 과정
1. tensorflow_yolo_tflite 라이브러리를 clone
1. tensorflow_yolo_tflite 경로를 PYTHONPATH 환경변수로 설정
1. core/config.py 의 C.YOLO.CLASSES 에 names 파일 경로 설정
1. darknet으로 train한 weight를 tensorflow 모델로 convert (./checkponts/yolo-416)

- detect.py 코드를 참고해서 detection하는 코드를 작성

# import

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
from PIL import Image

import tensorflow as tf
# tensorflow_yolo_tflite 의 model들
from core import utils
from core.yolov4 import filter_boxes

# 변수들 정의

In [3]:
IMAGE_SIZE = 416 #input image size
MODEL_PATH = 'tensorflow-yolov4-tflite/checkpoints/yolov4-416/'  # tensorflow keras 모델로 변환된 saved_model 경로
OUTPUT_PATH = './predict_results' #결과 이미지를 저장할 경로
if not os.path.isdir(OUTPUT_PATH):
    os.mkdir(OUTPUT_PATH)
# NMS에서 사용할 Threshold 값들 설정
IOU_THRESH = 0.5
CONFIDENCE_SCORE_THRESH = 0.3

# 추론할 이미지 로딩 및 전처리

In [42]:
IMAGE_PATH = 'tensorflow-yolov4-tflite/data/kite.jpg'

ori_image = cv2.cvtColor(cv2.imread(IMAGE_PATH), cv2.COLOR_BGR2RGB)

# 전처리 - RESIZE, 정규화, BATCH 축 추가
input_tensor = cv2.resize(ori_image, (IMAGE_SIZE, IMAGE_SIZE))
input_tensor = input_tensor.astype(np.float32)
input_tensor = input_tensor/255.0
input_tensor = input_tensor[np.newaxis, ...]
# tensflow Tensor 로 변환
input_tensor = tf.convert_to_tensor(input_tensor)

In [5]:
input_tensor.dtype, input_tensor.shape

(tf.float32, TensorShape([1, 416, 416, 3]))

# 모델 로드 & 추론

In [6]:
# 모델 로드
coco_model = tf.keras.models.load_model(MODEL_PATH)

In [7]:
coco_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 416, 416, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 416, 416, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 416, 416, 32) 128         conv2d[0][0]                     
__________________________________________________________________________________________________
tf.math.softplus (TFOpLambda)   (None, 416, 416, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

batch_normalization_105 (BatchN (None, 13, 13, 512)  2048        conv2d_107[0][0]                 
__________________________________________________________________________________________________
tf.nn.leaky_relu_33 (TFOpLambda (None, 13, 13, 512)  0           batch_normalization_105[0][0]    
__________________________________________________________________________________________________
conv2d_92 (Conv2D)              (None, 52, 52, 256)  294912      tf.nn.leaky_relu_19[0][0]        
__________________________________________________________________________________________________
conv2d_100 (Conv2D)             (None, 26, 26, 512)  1179648     tf.nn.leaky_relu_26[0][0]        
__________________________________________________________________________________________________
conv2d_108 (Conv2D)             (None, 13, 13, 1024) 4718592     tf.nn.leaky_relu_33[0][0]        
__________________________________________________________________________________________________
batch_norm

In [8]:
# 추론
pred = coco_model.predict(input_tensor)

In [11]:
print(type(pred), pred.dtype, pred.shape)
# (1, 45, 84) - (예측개수, bbox개수, bbox별 예측결과값)
# bbox별 예측결과값: 4: bbox좌표(cx,cy,w,h), 80: class별 확률

<class 'numpy.ndarray'> float32 (1, 45, 84)


# 후처리
- box좌표, class별 확률을 분리
- NMS 실행

In [12]:
pred_boxes = pred[:, :, 0:4] #예측한 bbox좌표
pred_conf = pred[:, :, 4:]   #예측한 class별 확률
print(pred_boxes.shape, pred_conf.shape)

(1, 45, 4) (1, 45, 80)


In [14]:
tf.reshape(pred_boxes, (tf.shape(pred_boxes)[0], -1, 1, 4)).shape

TensorShape([1, 45, 1, 4])

In [19]:
tf.reshape(pred_conf, (tf.shape(pred_conf)[0], -1, tf.shape(pred_conf)[-1])).shape

TensorShape([1, 45, 80])

In [20]:
# NMS 처리
boxes, scores, classes, valid_detections = tf.image.combined_non_max_suppression(
    boxes=tf.reshape(pred_boxes, (tf.shape(pred_boxes)[0], -1, 1, 4)), #bbox 좌표 추론값
    scores=tf.reshape(pred_conf, (tf.shape(pred_conf)[0], -1, tf.shape(pred_conf)[-1])), #class별 추론값
    max_output_size_per_class=50, #class별로 최대 몇개의 bbox를 찾을 것인지.
    max_total_size=50, # 전체 bbox를 몇개 찾을 것인지.
    iou_threshold=IOU_THRESH, #IoU 임계값.
    score_threshold=CONFIDENCE_SCORE_THRESH #CONFIDENCE SCORE 임계값 - 이거보다 낮은 확률로 예측된 것은 안나오게 한다.
)

In [45]:
# NMS 처리결과(tf.Tensor)를 ndarray로 변환
# boxes: bounding box의 좌표
# scores: 찾은 class의 확률
# classes: 찾은 class의 라벨
# valid_detections: 찾은 object의 개수
pred_bbox = [boxes.numpy(), scores.numpy(), classes.numpy(), valid_detections.numpy()]

In [46]:
# shape 확인
for arr in pred_bbox:
    print(arr.shape)

(1, 50, 4)
(1, 50)
(1, 50)
(1,)


In [47]:
# pred_bbox[0]
pred_bbox[3]

array([17])

# 추론결과를 사용
- valid_detections 만큼 정보를 조회해서 사용

In [48]:
for i in range(pred_bbox[3][0]):
    # 좌표, 확률, class
    print(f"{i}=",pred_bbox[0][0,i], pred_bbox[1][0,i], int(pred_bbox[2][0,i]))

0= [0.7725003  0.15819524 0.95587105 0.2001662 ] 0.99105513 0
1= [0.08883843 0.4350317  0.17153433 0.49754646] 0.98884565 33
2= [0.68207836 0.0840896  0.8491423  0.12258308] 0.9860108 0
3= [0.3822156  0.42745858 0.40842822 0.4442834 ] 0.8796148 33
4= [0.5421252  0.254808   0.560501   0.26386842] 0.8587331 0
5= [0.26256543 0.20689727 0.3109366  0.22676025] 0.8439843 33
6= [0.5600144  0.38356537 0.5853266  0.3956331 ] 0.78574175 0
7= [0.5680103  0.02381866 0.617023   0.04278604] 0.785423 0
8= [0.56479895 0.05980185 0.63085824 0.07934856] 0.76375955 0
9= [0.3772436  0.3463357  0.39720488 0.3593467 ] 0.7356637 33
10= [0.598063   0.13111815 0.64023805 0.14322905] 0.6882722 0
11= [0.43732578 0.8011486  0.47132567 0.81500345] 0.6840349 33
12= [0.41698185 0.2257757  0.44124863 0.24203326] 0.5575619 33
13= [0.5745783  0.39433923 0.59098613 0.4089731 ] 0.46349052 0
14= [0.50204057 0.8919314  0.5144351  0.8990725 ] 0.41316602 0
15= [0.41959247 0.5634138  0.43494883 0.570324  ] 0.4003401 33
16= [0

### bounding box
- 추론 결과를 이용해 직접그리기.
- 라이브러리에서 제공하는 함수

In [49]:
os.path.join(OUTPUT_PATH, f"result_{os.path.split(IMAGE_PATH)[1]}")

'./predict_results\\result_kite.jpg'

In [50]:
bbox_image = utils.draw_bbox(ori_image, pred_bbox)  #(원본이미지, [bbox좌표, scores, classes, 물체개수])
bbox_image = bbox_image.astype(np.uint8)
output_file_path = os.path.join(OUTPUT_PATH, f"result_{os.path.split(IMAGE_PATH)[1]}")

# draw_bbox(): RGB -> BGR
bbox_image = cv2.cvtColor(bbox_image, cv2.COLOR_RGB2BGR)
cv2.imwrite(output_file_path, bbox_image)

True

# Lion, Tiger detection 모델
- lion, tiger 학습했던 weights(yolov3_best.weight)를 tensorflow_yolo_tflite/pretrained_weights에 복사
- names 파일을 작성
    - data/classes/obj.names : lion, tiger 등록
------------------------주피터노트북 Restart------------------------  
- core/config.py
    - C.YOLO.CLASSES 변수의 경로를 obj.names로 변경
- yolov3_best.weight 를 tensorflow keras 모델로 convert
    - tensorflow_yolo_tflite/checkpoints/yolo_lion_tiger 에 저장
- 구글에서 사자, 호랑이 사진을 다운받은 뒤에 디텍션 처리.

# import

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
from PIL import Image

import tensorflow as tf
# tensorflow_yolo_tflite 의 model들
from core import utils
from core.yolov4 import filter_boxes

# 변수들 정의

In [2]:
os.getcwd()

'C:\\Users\\Playdata\\05_ObjectDetection\\lion_tiger_yolo_tensorflow'

In [4]:
# os.chdir("변경할 working directory경로")

In [5]:
IMAGE_SIZE = 416 #input image size
MODEL_PATH = 'tensorflow-yolov4-tflite/checkpoints/yolov4-lion_tiger/'  # tensorflow keras 모델로 변환된 saved_model 경로
OUTPUT_PATH = './predict_results' #결과 이미지를 저장할 경로
if not os.path.isdir(OUTPUT_PATH):
    os.mkdir(OUTPUT_PATH)
# NMS에서 사용할 Threshold 값들 설정
IOU_THRESH = 0.5
CONFIDENCE_SCORE_THRESH = 0.3

# 모델 로드

In [6]:
model = tf.keras.models.load_model(MODEL_PATH)

# 추론할 이미지 로드 및 전처리

In [32]:
IMAGE_PATH = 'test_data/lion2.jpg'

ori_image = cv2.cvtColor(cv2.imread(IMAGE_PATH), cv2.COLOR_BGR2RGB)

# 전처리 - RESIZE, 정규화, BATCH 축 추가
input_tensor = cv2.resize(ori_image, (IMAGE_SIZE, IMAGE_SIZE))
input_tensor = input_tensor.astype(np.float32)
input_tensor = input_tensor/255.0
input_tensor = input_tensor[np.newaxis, ...]
# tensflow Tensor 로 변환
input_tensor = tf.convert_to_tensor(input_tensor)

# 추론

In [33]:
pred = model.predict(input_tensor)

# 후처리

In [34]:
pred_boxes = pred[:, :, 0:4] #예측한 bbox좌표
pred_conf = pred[:, :, 4:]   #예측한 class별 확률

In [35]:
# NMS 처리
boxes, scores, classes, valid_detections = tf.image.combined_non_max_suppression(
    boxes=tf.reshape(pred_boxes, (tf.shape(pred_boxes)[0], -1, 1, 4)), #bbox 좌표 추론값
    scores=tf.reshape(pred_conf, (tf.shape(pred_conf)[0], -1, tf.shape(pred_conf)[-1])), #class별 추론값
    max_output_size_per_class=50, #class별로 최대 몇개의 bbox를 찾을 것인지.
    max_total_size=50, # 전체 bbox를 몇개 찾을 것인지.
    iou_threshold=IOU_THRESH, #IoU 임계값.
    score_threshold=CONFIDENCE_SCORE_THRESH #CONFIDENCE SCORE 임계값 - 이거보다 낮은 확률로 예측된 것은 안나오게 한다.
)

In [36]:
# 결과를 ndarray로 변환
pred_bbox = [boxes.numpy(), scores.numpy(), classes.numpy(), valid_detections.numpy()]

# 추론 결과 확인

In [37]:
# 좌표, class를 조회
h = ori_image.shape[0]
w = ori_image.shape[1]
for i in range(pred_bbox[3][0]): #valid_detections값 만큼 반복
    # 좌표, 확률, class
    print(f"{i}=",pred_bbox[0][0,i], pred_bbox[1][0,i], int(pred_bbox[2][0,i]))  #box좌표, 확률, label 
    #cx
    print("원래 좌표로 원복", pred_bbox[0][0,i,0]*w, pred_bbox[0][0,i,1]*h, pred_bbox[0][0,i,2]*w, pred_bbox[0][0,i,3]*h)

0= [0.09603288 0.02196429 1.         0.62174666] 0.96789664 0
원래 좌표로 원복 98.3376693725586 15.001613372936845 1024.0 424.65296828746796
1= [0.05530361 0.41867557 0.9903292  0.95071995] 0.95359 0
원래 좌표로 원복 56.630897521972656 285.9554154574871 1014.0971069335938 649.3417276144028


In [38]:
ori_image.shape

(683, 1024, 3)

In [39]:
bbox_image = utils.draw_bbox(ori_image, pred_bbox)  #(원본이미지, [bbox좌표, scores, classes, 물체개수])
bbox_image = bbox_image.astype(np.uint8)
output_file_path = os.path.join(OUTPUT_PATH, f"result_{os.path.split(IMAGE_PATH)[1]}")

# draw_bbox(): RGB -> BGR
bbox_image = cv2.cvtColor(bbox_image, cv2.COLOR_RGB2BGR)
cv2.imwrite(output_file_path, bbox_image)

True